# Source Mapping

Learn how to use bidirectional text ↔ structure mapping with `IntermediateRepresentation`.

In [1]:
from t_prompts import prompt

## Rendering and Source Maps

The `render()` method returns an `IntermediateRepresentation` with the final text and a source map that tracks which parts of the text came from which elements (both static and interpolated).

The `IntermediateRepresentation` serves as a bridge between structured prompts and their final output, enabling structured optimization and debugging.

In [2]:
name = "Alice"
age = "30"
p = prompt(t"Name: {name:n}, Age: {age:a}")

rendered = p.render()

print(f"Text: {rendered.text}")
print(f"\nSource map ({len(rendered.source_map)} spans):")
for span in rendered.source_map:
    print(f"  [{span.start}:{span.end}] key='{span.key}' -> \"{rendered.text[span.start:span.end]}\"")

Text: Name: Alice, Age: 30

Source map (4 spans):
  [0:6] key='0' -> "Name: "
  [6:11] key='n' -> "Alice"
  [11:18] key='1' -> ", Age: "
  [18:20] key='a' -> "30"


## Finding Source by Position

Use `get_span_at(pos)` to find which interpolation produced a specific position in the rendered text.

In [3]:
# Position 6 is in "Alice"
span = rendered.get_span_at(6)
print("Position 6:")
print(f"  Key: {span.key}")
print(f"  Span: [{span.start}:{span.end}]")
print(f"  Text: \"{rendered.text[span.start:span.end]}\"")

# Position 18 is in "30"
span = rendered.get_span_at(18)
print("\nPosition 18:")
print(f"  Key: {span.key}")
print(f"  Span: [{span.start}:{span.end}]")
print(f"  Text: \"{rendered.text[span.start:span.end]}\"")

Position 6:
  Key: n
  Span: [6:11]
  Text: "Alice"

Position 18:
  Key: a
  Span: [18:20]
  Text: "30"


## Finding Position by Key

Use `get_span_for_key(key)` to find where a specific key was rendered in the text.

In [4]:
# Find where 'n' (name) was rendered
span_n = rendered.get_span_for_key('n')
print("Key 'n':")
print(f"  Span: [{span_n.start}:{span_n.end}]")
print(f"  Text: \"{rendered.text[span_n.start:span_n.end]}\"")

# Find where 'a' (age) was rendered
span_a = rendered.get_span_for_key('a')
print("\nKey 'a':")
print(f"  Span: [{span_a.start}:{span_a.end}]")
print(f"  Text: \"{rendered.text[span_a.start:span_a.end]}\"")

Key 'n':
  Span: [6:11]
  Text: "Alice"

Key 'a':
  Span: [18:20]
  Text: "30"


## Source Mapping for Nested Prompts

Source maps work recursively for nested prompts. Each span includes a `path` tuple showing the nesting hierarchy.

In [5]:
greeting = "Hello"
name = "Alice"  # Re-use from earlier example
inner = prompt(t"{greeting:g}, {name:n}!")
outer = prompt(t"Message: {inner:msg}")

rendered_nested = outer.render()

print(f"Text: {rendered_nested.text}")
print("\nSource map with paths:")
for span in rendered_nested.source_map:
    print(f"  path={span.path} key='{span.key}' -> \"{rendered_nested.text[span.start:span.end]}\"")

Text: Message: Hello, Alice!

Source map with paths:
  path=() key='0' -> "Message: "
  path=('msg',) key='g' -> "Hello"
  path=('msg',) key='1' -> ", "
  path=('msg',) key='n' -> "Alice"
  path=('msg',) key='2' -> "!"


In [6]:
from t_prompts import Element, Static, StructuredInterpolation

value = "test"
p = prompt(t"prefix {value:v} suffix")

print(f"Prompt has {len(p.elements)} elements:\n")
for i, elem in enumerate(p.elements):
    if isinstance(elem, Static):
        print(f"{i}. Static(key={elem.key}, index={elem.index}, value={elem.value!r})")
    elif isinstance(elem, StructuredInterpolation):
        print(f"{i}. Interpolation(key={elem.key!r}, index={elem.index}, value={elem.value!r})")

    # Both are Elements
    assert isinstance(elem, Element)

Prompt has 3 elements:

0. Static(key=0, index=0, value='prefix ')
1. Interpolation(key='v', index=1, value='test')
2. Static(key=1, index=2, value=' suffix')


### Elements: Unified Access to Structure

The `elements` property provides unified access to **all** components of a prompt (both static and interpolations):

In [7]:
# Get static text span by its index (position in template strings)
static_span = rendered.get_static_span(0)  # First static segment "Hello "
print("Static segment 0:")
print(f"  Text: \"{rendered.text[static_span.start:static_span.end]}\"")
print(f"  Span: [{static_span.start}:{static_span.end}]")

# Get interpolation span by its key
interp_span = rendered.get_interpolation_span('n')
print("\nInterpolation 'n':")
print(f"  Text: \"{rendered.text[interp_span.start:interp_span.end]}\"")
print(f"  Span: [{interp_span.start}:{interp_span.end}]")

# Get second static segment "!"
static_span2 = rendered.get_static_span(1)
print("\nStatic segment 1:")
print(f"  Text: \"{rendered.text[static_span2.start:static_span2.end]}\"")
print(f"  Span: [{static_span2.start}:{static_span2.end}]")

Static segment 0:
  Text: "Name: "
  Span: [0:6]

Interpolation 'n':
  Text: "Alice"
  Span: [6:11]

Static segment 1:
  Text: ", Age: "
  Span: [11:18]


### Helper Methods for Static Text

Use `get_static_span()` and `get_interpolation_span()` to specifically query static or interpolation spans:

Notice how **all** parts of the text are now mapped:
- Static text segments have `element_type="static"` and integer keys (their position in the template)
- Interpolations have `element_type="interpolation"` and string keys (from format specs)

This enables complete bidirectional mapping for the entire rendered text!

In [8]:
name = "Alice"
p = prompt(t"Hello {name:n}!")

rendered = p.render()

print(f"Text: {rendered.text}")
print("\nComplete source map (including static text):")
for span in rendered.source_map:
    text_slice = rendered.text[span.start:span.end]
    print(f"  [{span.start}:{span.end}] {span.element_type:13s} key={span.key!r:5s} -> \"{text_slice}\"")

Text: Hello Alice!

Complete source map (including static text):
  [0:6] static        key=0     -> "Hello "
  [6:11] interpolation key='n'   -> "Alice"
  [11:12] static        key=1     -> "!"


## Source Mapping for Static Text (New in 0.4.0)

As of version 0.4.0, source maps include **all** text in the rendered output, not just interpolations. This includes the static literal text between interpolations.

Each span now has an `element_type` field that distinguishes between `"static"` and `"interpolation"` spans.

## Accessing the Original Prompt

The `IntermediateRepresentation` keeps a reference to the original `StructuredPrompt`.

In [9]:
# Access the source prompt
assert rendered_nested.source_prompt is outer

# Use it to get metadata
span = rendered_nested.get_span_for_key('msg')
node = rendered_nested.source_prompt['msg']
print("Key 'msg':")
print(f"  Expression: {node.expression}")
print(f"  Type: {type(node.value).__name__}")

Key 'msg':
  Expression: inner
  Type: StructuredPrompt


## Use Case: Highlighting in a UI

Source maps enable building UIs that highlight which parts of a rendered prompt came from which variables.

In [10]:
def highlight_key(rendered, key, marker='**'):
    """Highlight a specific key in the rendered text."""
    span = rendered.get_span_for_key(key)
    text = rendered.text
    return text[:span.start] + marker + text[span.start:span.end] + marker + text[span.end:]

city = "Paris"
country = "France"
p = prompt(t"{city:c} is the capital of {country:co}.")
r = p.render()

print("Original:")
print(r.text)
print("\nHighlight 'c':")
print(highlight_key(r, 'c'))
print("\nHighlight 'co':")
print(highlight_key(r, 'co'))

Original:
Paris is the capital of France.

Highlight 'c':
**Paris** is the capital of France.

Highlight 'co':
Paris is the capital of **France**.


## Use Case: Error Attribution

If an LLM returns an error pointing to a character position, you can trace it back to the source variable.

In [11]:
def trace_error_to_source(rendered, error_pos):
    """Trace an error position back to its source variable."""
    try:
        span = rendered.get_span_at(error_pos)
        if span is None:
            # Position is in a literal string, not an interpolation
            return None
        node = rendered.source_prompt[span.key]
        return {
            'key': span.key,
            'expression': node.expression,
            'value': node.value,
            'text_span': rendered.text[span.start:span.end]
        }
    except (KeyError, AttributeError):
        return None

# Simulate an error at position 18 (in "30")
error_info = trace_error_to_source(rendered, 18)
if error_info:
    print("Error at position 18 traced to:")
    print(f"  Key: {error_info['key']}")
    print(f"  Expression: {error_info['expression']}")
    print(f"  Value: {error_info['value']}")
    print(f"  Text: \"{error_info['text_span']}\"")
else:
    print("Position is not in an interpolation.")

Position is not in an interpolation.
